# Chapter 33: Monitoring and Maintenance

---

Deploying a smart contract is not the end of the journey—it's the beginning of a new phase: operations. Once your contract is live on mainnet, you must actively monitor its health, security, and performance. Users depend on it, and any issue can have real financial consequences. This chapter covers the essential aspects of post-deployment monitoring and maintenance: tracking events and transactions, detecting anomalies, planning for incidents, and managing upgrades. By implementing robust monitoring and having a clear incident response plan, you can ensure your contract remains secure and reliable over its lifetime.

---

## 33.1 Monitoring Smart Contracts

Effective monitoring gives you visibility into your contract's behavior and alerts you to potential problems before they escalate.

### 33.1.1 Event Monitoring

Events are the primary way smart contracts communicate with the outside world. By monitoring events, you can track important state changes, user activity, and potential issues.

**What to monitor:**
- **Key business events**: Transfers, deposits, withdrawals, liquidations.
- **Admin events**: Ownership changes, parameter updates, pauses.
- **Error events**: Custom error events (if you emit them) can indicate failed attempts or malicious activity.

**Tools for event monitoring:**

- **The Graph**: Build a subgraph to index events and query them via GraphQL. You can also subscribe to real-time updates.
- **Tenderly**: Provides a dashboard for event monitoring, with alerts and transaction simulations.
- **OpenZeppelin Defender Sentinels**: Create automated alerts based on event conditions.
- **Custom scripts**: Use ethers.js/web3.js to listen for events and send notifications.

**Example: Listening to Transfer events with ethers.js**
```javascript
const { ethers } = require("ethers");

const provider = new ethers.providers.WebSocketProvider(process.env.WSS_URL);
const contract = new ethers.Contract(contractAddress, abi, provider);

contract.on("Transfer", (from, to, value, event) => {
  console.log(`Transfer: ${from} -> ${to}, value: ${ethers.utils.formatEther(value)} ETH`);
  
  // Check for large transfers
  if (value.gt(ethers.utils.parseEther("1000"))) {
    sendAlert(`Large transfer detected: ${value} ETH`);
  }
});
```

**Setting up alerts with Defender Sentinel:**
1. Create a Sentinel in Defender.
2. Choose condition type: "Event" or "Function".
3. Specify event signature and filter (e.g., `value > 1000 ether`).
4. Choose notification channels: email, Slack, Discord, Telegram.

### 33.1.2 Transaction Monitoring

Monitoring transactions helps you detect failures, gas issues, and suspicious patterns.

**Key metrics:**
- **Success rate**: Percentage of transactions that succeed vs. revert. A sudden increase in reverts may indicate a bug or attack.
- **Gas used**: Unusually high gas consumption could signal inefficiency or an attack (e.g., gas griefing).
- **From addresses**: Monitor for addresses that interact frequently or in suspicious patterns.
- **MEV activity**: Look for front-running or sandwich attacks.

**Tools:**
- **Etherscan API**: Query transaction history for your contract.
- **Tenderly**: Provides transaction tracing and debugging.
- **Dune Analytics**: Build dashboards to visualize transaction metrics.
- **Custom analytics**: Index transactions into a database for analysis.

**Example: Monitoring revert rate with a script**
```javascript
const { ethers } = require("ethers");

async function monitorReverts() {
  const provider = new ethers.providers.JsonRpcProvider(process.env.RPC_URL);
  const latestBlock = await provider.getBlockNumber();
  const fromBlock = latestBlock - 1000; // last 1000 blocks

  const logs = await provider.getLogs({
    address: contractAddress,
    fromBlock,
    toBlock: latestBlock
  });

  // Count transactions by parsing receipts
  const txHashes = [...new Set(logs.map(l => l.transactionHash))];
  let success = 0, fail = 0;

  for (const hash of txHashes) {
    const receipt = await provider.getTransactionReceipt(hash);
    if (receipt.status === 1) success++; else fail++;
  }

  const revertRate = fail / (success + fail);
  if (revertRate > 0.1) {
    sendAlert(`High revert rate: ${(revertRate * 100).toFixed(2)}%`);
  }
}
```

### 33.1.3 Gas Price Monitoring

Gas prices fluctuate based on network demand. Monitoring gas helps you optimize transaction costs and detect anomalies.

**What to monitor:**
- **Gas price trends**: Are users overpaying? Could you suggest optimal gas prices?
- **Gas spikes**: Sudden spikes could indicate network congestion or MEV activity.
- **Contract gas usage**: Track average gas used per function to identify inefficiencies.

**Tools:**
- **Etherscan gas tracker**: API for current gas prices.
- **Gas station network**: Third-party APIs (e.g., ETH Gas Station).
- **Custom monitoring**: Poll gas prices and alert on spikes.

**Example: Alerting on high gas prices**
```javascript
const axios = require('axios');

async function checkGasPrice() {
  const response = await axios.get('https://api.etherscan.io/api?module=gastracker&action=gasoracle&apikey=YOUR_KEY');
  const safeGas = response.data.result.SafeGasPrice;
  const proposeGas = response.data.result.ProposeGasPrice;
  const fastGas = response.data.result.FastGasPrice;

  if (fastGas > 200) { // 200 gwei threshold
    sendAlert(`High gas price: ${fastGas} gwei`);
  }
}
```

---

## 33.2 Security Monitoring

Beyond operational metrics, you must monitor for security threats.

### 33.2.1 Anomaly Detection

Look for patterns that deviate from normal behavior:

- **Unusual transaction volume**: Sudden spikes in interactions.
- **Large value movements**: Transfers exceeding typical amounts.
- **Repeated failures**: Multiple failed attempts from same address (possible attack probing).
- **Unusual function calls**: Invocations of rarely-used functions.

**Implementation ideas:**
- Set thresholds based on historical data.
- Use machine learning (advanced) to detect anomalies.
- Monitor for known attack patterns (e.g., flash loan signatures).

**Example: Detecting flash loan attempts**
```javascript
// Flash loans often involve calling multiple protocols in one transaction
// Look for transactions that interact with multiple DeFi contracts
async function detectFlashLoan(txHash) {
  const tx = await provider.getTransaction(txHash);
  const receipt = await provider.getTransactionReceipt(txHash);
  
  // If transaction calls many different contracts, it might be a flash loan
  const uniqueContracts = new Set();
  receipt.logs.forEach(log => uniqueContracts.add(log.address));
  
  if (uniqueContracts.size > 5) {
    sendAlert(`Possible flash loan attack in tx: ${txHash}`);
  }
}
```

### 33.2.2 Governance Monitoring

If your contract is governed by a DAO or multi-sig, monitor governance activity:

- **Proposal creation**: Track new proposals.
- **Voting activity**: Monitor vote counts and participation.
- **Timelock events**: Track scheduled and executed transactions.
- **Ownership changes**: Any change in admin roles.

**Tools:**
- **OpenZeppelin Defender**: Monitor and automate governance actions.
- **Tally**: Governance analytics for DAOs.
- **Custom subgraphs**: Index governance events.

**Example: Monitoring timelock execution**
```javascript
timelock.on("CallExecuted", (id, index, target, value, data) => {
  console.log(`Timelock executed: target ${target}, value ${value}`);
  // Alert on high-value executions
  if (value.gt(ethers.utils.parseEther("100"))) {
    sendAlert(`High-value timelock execution: ${value} ETH`);
  }
});
```

---

## 33.3 Incident Response Planning

Despite all precautions, incidents can happen. Having a well-defined incident response plan minimizes damage and confusion.

### 33.3.1 Emergency Procedures

Define clear steps for different types of incidents:

| Incident Type | Immediate Actions | Responsible Parties |
|---------------|-------------------|---------------------|
| **Critical vulnerability** | Pause contract (if pausable), notify community, prepare fix | Dev team, security lead |
| **Oracle failure** | Switch to backup oracle, halt dependent functions | Operations team |
| **Large-scale attack** | Pause, analyze attack, coordinate with validators/MEV searchers | Security team, advisors |
| **Key compromise** | Rotate keys, transfer ownership to secure address | Multi-sig signers |

**Example: Emergency pause script**
```javascript
// This script would be run by a multi-sig signer
async function emergencyPause() {
  const safe = new ethers.Contract(safeAddress, safeABI, signer);
  const txData = contract.interface.encodeFunctionData("pause", []);
  await safe.execTransaction(contractAddress, 0, txData, 0, 0, 0, 0, 0, 0);
}
```

### 33.3.2 Communication Plans

During an incident, transparent communication is crucial to maintain user trust.

**Communication channels:**
- **Official website / blog**: For major announcements.
- **Twitter / Discord / Telegram**: Real-time updates.
- **Email list**: For registered users.
- **Bug bounty platform**: If applicable.

**Communication timeline:**
- **Within minutes**: Acknowledge issue, state that team is investigating.
- **Within hours**: Provide initial findings, actions taken (e.g., "contract paused").
- **Within days**: Post-mortem with root cause, impact, and prevention measures.

**Template for incident communication:**
```
🚨 Incident Update

We are aware of unusual activity on [contract]. The contract has been paused as a precaution. Our team is investigating. We will provide updates within the next hour.

- Time: [timestamp]
- Status: Investigating
- Action taken: Paused contract

Stay tuned.
```

### 33.3.3 Recovery Strategies

Depending on the incident, you may need to recover funds or restore functionality.

**Common recovery strategies:**

1. **Upgrade to fixed contract** (if upgradeable): Deploy new implementation and point proxy.
2. **Manual refunds**: If funds are stuck, you may need to deploy a recovery contract.
3. **Forking**: In extreme cases (rare), the community may fork the chain to reverse transactions (e.g., DAO fork).

**Example: Recovering stuck funds**
```solidity
// Recovery contract (only callable by owner)
function recoverTokens(address token, address to, uint amount) external onlyOwner {
    IERC20(token).transfer(to, amount);
}
```

**Post-recovery:**
- Perform a thorough post-mortem.
- Implement additional safeguards (e.g., rate limits, better monitoring).
- Consider compensating affected users (depending on circumstances).

---

## 33.4 Upgrades and Patches

If your contract is upgradeable, you have the ability to fix bugs and add features. However, upgrades must be handled carefully.

### 33.4.1 Planning Upgrades

Before upgrading:

- **Identify the need**: Bug fix, new feature, optimization, regulatory compliance.
- **Assess impact**: Will the upgrade change existing functionality? Could it break integrations?
- **Develop and test**: Write new implementation, test thoroughly (including storage compatibility).
- **Security review**: Even if minor, consider a quick audit or internal review.

**Storage compatibility check:** Ensure new implementation does not modify existing storage layout. Use OpenZeppelin Upgrades plugin to validate.

```bash
npx hardhat run scripts/check-upgrade.js
```

### 33.4.2 Communicating Changes

Before executing an upgrade, communicate with users and integrators:

- **Publish upgrade proposal** (if DAO-governed).
- **Explain changes** in plain language.
- **Provide timeline**: When will upgrade happen, how long will it take, is there downtime?
- **Encourage testing** on testnet first.

**Example upgrade announcement:**
```
📢 Upgrade Proposal: MyContract v2

We are proposing an upgrade to MyContract to add support for a new asset type. The upgrade is non-disruptive (proxy remains same address). Changes:
- Add `mintNewAsset` function
- Fix gas inefficiency in `transfer`

Timeline: Proposal voting ends Oct 15, if passed, upgrade will execute Oct 16 after timelock.

Please review the code on GitHub: [link]
```

### 33.4.3 Migration Strategies

Sometimes an upgrade isn't enough—you may need to migrate to an entirely new contract (e.g., if storage layout is incompatible). In that case, you need a migration plan.

**Options:**

1. **Snapshot migration**: Take a snapshot of users' balances at a block, and let them claim on new contract.
2. **One-way bridge**: Users manually migrate by sending tokens to old contract and receiving new ones (with a 1:1 ratio).
3. **Automatic migration**: Deploy a migration contract that moves all state to new contract atomically (complex, gas-intensive).

**Example: Snapshot claim contract**
```solidity
contract MigratedToken {
    mapping(address => uint) public balances;
    mapping(address => bool) public claimed;

    constructor(address oldToken, uint blockNumber) {
        // Take snapshot at blockNumber (using archival node or indexer)
        // This is simplified; in practice you'd use merkle proofs
    }

    function claim() external {
        require(!claimed[msg.sender], "Already claimed");
        uint amount = balances[msg.sender];
        claimed[msg.sender] = true;
        _mint(msg.sender, amount);
    }
}
```

**Communication:** Clearly explain the migration process, deadlines, and any actions required from users.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 33 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Post-deployment monitoring and maintenance are essential.      │
│                                                                 │
│  Monitoring:                                                    │
│    • Events: track key business events and set alerts.         │
│    • Transactions: monitor success rates, gas usage, patterns. │
│    • Gas prices: detect spikes and optimize costs.             │
│                                                                 │
│  Security monitoring:                                           │
│    • Anomaly detection (volume, value, patterns).              │
│    • Governance monitoring (proposals, voting, timelock).      │
│                                                                 │
│  Incident response:                                             │
│    • Define emergency procedures (pause, notify, analyze).     │
│    • Communicate clearly and promptly.                         │
│    • Plan recovery strategies (upgrade, refunds).              │
│                                                                 │
│  Upgrades:                                                      │
│    • Plan, test, and communicate upgrades.                     │
│    • Use storage compatibility checks.                         │
│    • For major changes, plan migrations (snapshot, claims).    │
│                                                                 │
│  Proactive monitoring and a solid incident response plan       │
│  build user trust and protect your protocol.                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 34 – Legal and Regulatory Considerations. We'll explore the legal landscape for blockchain projects, including securities regulations, KYC/AML requirements, tax implications, and jurisdictional differences.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='32. deploying_to_production.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='34. legal_and_regulatory_considerations.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
